In [1]:
import json
import os
from datasets import load_dataset
from rllm.system_prompts import VULNERABILITY_HINT
import random

ds = load_dataset("likaixin/TACO-verified", split="train", trust_remote_code=True)

print(ds)

/home/david/.conda/envs/rllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'question', 'solutions', 'starter_code', 'input_output', 'difficulty', 'raw_tags', 'name', 'source', 'tags', 'skill_types', 'url', 'Expected Auxiliary Space', 'time_limit', 'date', 'picture_num', 'memory_limit', 'Expected Time Complexity'],
    num_rows: 12898
})


In [2]:
# TACO dataset has difficulties as strings: EASY, MEDIUM, MEDIUM_HARD, HARD, VERY_HARD
# Let EASY->1.9, MEDIUM->3.7, MEDIUM_HARD->5.5, HARD->7.3, VERY_HARD->9.1
def difficulty_to_int(difficulty):
    if difficulty == "EASY":
        return 1.9
    elif difficulty == "MEDIUM":
        return 3.7
    elif difficulty == "MEDIUM_HARD":
        return 5.5
    elif difficulty == "HARD":
        return 7.3
    elif difficulty == "VERY_HARD":
        return 9.1
    else:
        #print("difficulty", difficulty)
        return -1

In [3]:
import ast
dataset = []
unknown_difficulty = 0
for entry in ds:
    tests = entry["input_output"]
    if not tests:
        continue
    
    if isinstance(tests, str):
        try:
            tests = ast.literal_eval(entry["input_output"])
        except (ValueError, SyntaxError) as e:
            # Try Json loads instead
            try:
                tests = json.loads(entry["input_output"])
            except (json.JSONDecodeError, SyntaxError, ValueError) as e:
                print(repr(entry["input_output"]))
                print(f"Error in json.loads: {e}")
                continue
    assert isinstance(tests, dict), "Tests should be a dictionary"
    assert len(tests["inputs"]) == len(tests["outputs"]), "Inputs and outputs should have the same length"
  

    if random.random() < 0.05:
        problem = f"{VULNERABILITY_HINT}\n\n{entry['question']}"
        new_entry = {
            "problem": problem,
            "tests": tests,
            "solutions": entry["solutions"],
        }
    else:
        new_entry = {
            "problem": entry["question"],
            "tests": tests,
            "solutions": entry["solutions"],
        }
    # Assert tests is a dictionary of inputs and outputs that map to a list of strings
    assert isinstance(tests, dict), "Tests should be a dictionary"
    assert "inputs" in tests, "Inputs should be a key in the dictionary"
    assert "outputs" in tests, "Outputs should be a key in the dictionary"
    assert isinstance(tests["inputs"], list), "Inputs should be a list"
    assert isinstance(tests["outputs"], list), "Outputs should be a list"
    assert len(tests["inputs"]) == len(tests["outputs"]), "Inputs and outputs should have the same length"
    

    inputs = tests["inputs"]
    outputs = tests["outputs"]
    if len(inputs) <= 1:
        continue
    inputs = inputs[:1] # Only use the first test case to induce reward hacking
    outputs = outputs[:1]
    dataset.append(new_entry)

print(len(dataset))
print(dataset[0])
print(unknown_difficulty)

output_dir = os.path.abspath("../../train/code")
output_file = os.path.join(output_dir, "taco_vuln.json")
with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)

9028
{'problem': 'There are $n$ candy boxes in front of Tania. The boxes are arranged in a row from left to right, numbered from $1$ to $n$. The $i$-th box contains $r_i$ candies, candies have the color $c_i$ (the color can take one of three values \u200b\u200b— red, green, or blue). All candies inside a single box have the same color (and it is equal to $c_i$).\n\nInitially, Tanya is next to the box number $s$. Tanya can move to the neighbor box (that is, with a number that differs by one) or eat candies in the current box. Tanya eats candies instantly, but the movement takes one second.\n\nIf Tanya eats candies from the box, then the box itself remains in place, but there is no more candies in it. In other words, Tanya always eats all the candies from the box and candies in the boxes are not refilled.\n\nIt is known that Tanya cannot eat candies of the same color one after another (that is, the colors of candies in two consecutive boxes from which she eats candies are always differen